In [43]:
# Import libraries
import requests
import pandas as pd
from datetime import datetime
import pytz
import sqlalchemy


# Set variables for the API, an empty list for weathers, and the cities list
API_key = "API Key"
weathers = []
cities =  ['berlin', 'paris', 'madrid']
# Request the weathers from the API
for city in cities:
  weathers.append(requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric").json())

# Function for getting the data for the Weather
def get_weather_loop(cities):


  # Establish the timezone and the time
  tz = pytz.timezone("Europe/Berlin")
  now = datetime.now().astimezone(tz)

  # Create a blank dictionary with the information to be requested
  weather_dict = {'city': [],
                  'country': [],
                  'forecast_time': [],
                  'outlook': [],
                  'detailed_outlook': [],
                  'temperature': [],
                  'temperature_feels_like': [],
                  'clouds': [],
                  'rain': [],
                  'snow': [],
                  'wind_speed': [],
                  'wind_deg': [],
                  'humidity': [],
                  'pressure': []}


  # Create a loop to get the json from the responses
  for city in cities:
    url =  (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    weather_json = response.json()



    # Create a loop to get the necesary information and store them into the dictionary
    for i in weather_json['list']:
      weather_dict['city'].append(weather_json['city']['name'])
      weather_dict['country'].append(weather_json['city']['country'])
      weather_dict['forecast_time'].append(i['dt_txt'])
      weather_dict['outlook'].append(i['weather'][0]['main'])
      weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
      weather_dict['temperature'].append(i['main']['temp'])
      weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
      weather_dict['clouds'].append(i['clouds']['all'])
      # Troubleshooting when there is no information
      try:
        weather_dict['rain'].append(i['rain']['3h'])
      except:
        weather_dict['rain'].append('0')
      try:
        weather_dict['snow'].append(i['snow']['3h'])
      except:
        weather_dict['snow'].append('0')
      weather_dict['wind_speed'].append(i['wind']['speed'])
      weather_dict['wind_deg'].append(i['wind']['deg'])
      weather_dict['humidity'].append(i['main']['humidity'])
      weather_dict['pressure'].append(i['main']['pressure'])

  return pd.DataFrame(weather_dict)

cities_weather = get_weather_loop(cities)
# Changing the names of the cities for an id
capitalized_cities = [city.capitalize() for city in cities]
cities_df = pd.DataFrame(capitalized_cities, columns=['city'])
cities_df.index =cities_df.index + 1
cities_weather['city_id'] = cities_weather['city_id'].map(cities_df.reset_index().set_index('city')['index'])
# Cleaning the data
cities_weather.drop('country', axis=1, inplace=True)
cities_weather.rename(columns={'city': 'city_id'}, inplace=True)
cities_weather['forecast_time'] = pd.to_datetime(cities_weather['forecast_time'])

# Copying the dataframe to MySQL
schema="mydb" 
host="127.0.0.1"
user="root"
password="*******"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

cities_weather.to_sql('weather', 
               if_exists='append',
               con=con,
               index=False)


,city_id,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure
0,1,2023-08-09 09:00:00,Clouds,scattered clouds,17.30,16.81,40,0,0,4.39,267,66,1010
1,1,2023-08-09 12:00:00,Clouds,broken clouds,18.35,17.76,51,0,0,4.48,271,58,1011
2,1,2023-08-09 15:00:00,Rain,light rain,18.43,17.74,69,0.14,0,4.36,284,54,1013
3,1,2023-08-09 18:00:00,Clouds,broken clouds,17.77,17.06,62,0,0,3.04,278,56,1016
4,1,2023-08-09 21:00:00,Clouds,scattered clouds,14.80,14.19,47,0,0,2.43,282,71,1018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,3,2023-08-13 18:00:00,Clouds,few clouds,35.34,32.61,16,0,0,7.61,245,9,1009
116,3,2023-08-13 21:00:00,Clouds,overcast clouds,31.31,29.18,100,0,0,4.61,255,12,1010
117,3,2023-08-14 00:00:00,Clouds,overcast clouds,28.38,26.92,100,0,0,2.69,215,15,1011
118,3,2023-08-14 03:00:00,Clouds,overcast clouds,25.70,24.80,99,0,0,2.26,208,18,1011
